In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

#xr.set_options(display_style='html')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
#!/usr/bin/env python
from __future__ import print_function
import requests
import xml.etree.ElementTree as ET
import numpy

In [3]:
# Author: Unknown
# I got the original version from a word document published by ESGF
# https://docs.google.com/document/d/1pxz1Kd3JHfFp8vR2JCVBfApbsHmbUQQstifhGNdc6U0/edit?usp=sharing

# API AT: https://github.com/ESGF/esgf.github.io/wiki/ESGF_Search_REST_API#results-pagination

def esgf_search(server="https://esgf-node.llnl.gov/esg-search/search",
                files_type="OPENDAP", local_node=True, project="CMIP6",
                verbose=False, format="application%2Fsolr%2Bjson",
                use_csrf=False, **search):
    client = requests.session()
    payload = search
    payload["project"] = project
    payload["type"]= "File"
    if local_node:
        payload["distrib"] = "false"
    if use_csrf:
        client.get(server)
        if 'csrftoken' in client.cookies:
            # Django 1.6 and up
            csrftoken = client.cookies['csrftoken']
        else:
            # older versions
            csrftoken = client.cookies['csrf']
        payload["csrfmiddlewaretoken"] = csrftoken

    payload["format"] = format

    offset = 0
    numFound = 10000
    all_files = []
    files_type = files_type.upper()
    while offset < numFound:
        payload["offset"] = offset
        url_keys = [] 
        for k in payload:
            url_keys += ["{}={}".format(k, payload[k])]

        url = "{}/?{}".format(server, "&".join(url_keys))
        print(url)
        r = client.get(url)
        r.raise_for_status()
        resp = r.json()["response"]
        numFound = int(resp["numFound"])
        resp = resp["docs"]
        offset += len(resp)
        for d in resp:
            if verbose:
                for k in d:
                    print("{}: {}".format(k,d[k]))
            url = d["url"]
            for f in d["url"]:
                sp = f.split("|")
                if sp[-1] == files_type:
                    all_files.append(sp[0].split(".html")[0])
    return sorted(all_files)

In [4]:
result = esgf_search(activity_id='ScenarioMIP', table_id='Amon', variable_id='tas', experiment_id='ssp585', 
                     institution_id="NCAR", source_id="CESM2", member_id="r10i1p1f1")
result

https://esgf-node.llnl.gov/esg-search/search/?activity_id=ScenarioMIP&table_id=Amon&variable_id=tas&experiment_id=ssp585&institution_id=NCAR&source_id=CESM2&member_id=r10i1p1f1&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=0


['http://esgf-data.ucar.edu/thredds/dodsC/esg_dataroot/CMIP6/ScenarioMIP/NCAR/CESM2/ssp585/r10i1p1f1/Amon/tas/gn/v20200528/tas_Amon_CESM2_ssp585_r10i1p1f1_gn_201501-206412.nc',
 'http://esgf-data.ucar.edu/thredds/dodsC/esg_dataroot/CMIP6/ScenarioMIP/NCAR/CESM2/ssp585/r10i1p1f1/Amon/tas/gn/v20200528/tas_Amon_CESM2_ssp585_r10i1p1f1_gn_206501-210012.nc',
 'http://esgf-data3.diasjp.net/thredds/dodsC/esg_dataroot/CMIP6/ScenarioMIP/NCAR/CESM2/ssp585/r10i1p1f1/Amon/tas/gn/v20200528/tas_Amon_CESM2_ssp585_r10i1p1f1_gn_201501-206412.nc',
 'http://esgf-data3.diasjp.net/thredds/dodsC/esg_dataroot/CMIP6/ScenarioMIP/NCAR/CESM2/ssp585/r10i1p1f1/Amon/tas/gn/v20200528/tas_Amon_CESM2_ssp585_r10i1p1f1_gn_206501-210012.nc',
 'https://esgf-data1.llnl.gov/thredds/dodsC/css03_data/CMIP6/ScenarioMIP/NCAR/CESM2/ssp585/r10i1p1f1/Amon/tas/gn/v20200528/tas_Amon_CESM2_ssp585_r10i1p1f1_gn_201501-206412.nc',
 'https://esgf-data1.llnl.gov/thredds/dodsC/css03_data/CMIP6/ScenarioMIP/NCAR/CESM2/ssp585/r10i1p1f1/Amon/tas

In [5]:
# there are mulitple sources of the same data--need to pick one
files_to_open = result[0:2]

ds = xr.open_mfdataset(files_to_open, concat_dim='time')
ds

C:\Users\tsun\Documents\Python3\lib\site-packages\xarray\conventions.py:394: SerializationWarning: variable 'tas' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  stack_char_dim=stack_char_dim)


<xarray.Dataset>
Dimensions:    (lat: 192, lon: 288, nbnd: 2, time: 1032)
Coordinates:
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time       (time) object 2015-01-15 12:00:00 ... 2100-12-15 12:00:00
Dimensions without coordinates: nbnd
Data variables:
    time_bnds  (time, nbnd) object dask.array<shape=(1032, 2), chunksize=(600, 2)>
    lat_bnds   (time, lat, nbnd) float64 dask.array<shape=(1032, 192, 2), chunksize=(600, 192, 2)>
    lon_bnds   (time, lon, nbnd) float64 dask.array<shape=(1032, 288, 2), chunksize=(600, 288, 2)>
    tas        (time, lat, lon) float32 dask.array<shape=(1032, 192, 288), chunksize=(600, 192, 288)>
Attributes:
    Conventions:                     CF-1.7 CMIP-6.2
    activity_id:                     ScenarioMIP
    branch_method:                   standard
    branch_time_in_child:            735110.0
    branch_time_in_parent:           73511

In [6]:
files_area = esgf_search(variable_id='areacella', activity_id='ScenarioMIP',
                         experiment_id='ssp585', institution_id="NCAR", source_id="CESM2")
ds_area = xr.open_dataset(files_area[0])
ds_area

https://esgf-node.llnl.gov/esg-search/search/?variable_id=areacella&activity_id=ScenarioMIP&experiment_id=ssp585&institution_id=NCAR&source_id=CESM2&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=0


C:\Users\tsun\Documents\Python3\lib\site-packages\xarray\conventions.py:394: SerializationWarning: variable 'areacella' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  stack_char_dim=stack_char_dim)


<xarray.Dataset>
Dimensions:    (lat: 192, lon: 288, nbnd: 2)
Coordinates:
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Dimensions without coordinates: nbnd
Data variables:
    lat_bnds   (lat, nbnd) float64 ...
    lon_bnds   (lon, nbnd) float64 ...
    areacella  (lat, lon) float32 ...
Attributes:
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            ScenarioMIP
    branch_method:          standard
    branch_time_in_child:   735110.0
    branch_time_in_parent:  735110.0
    case_id:                1733
    cesm_casename:          b.e21.BSSP585cmip6.f09_g17.CMIP6-SSP5-8.5.101
    contact:                cesm_cmip6@ucar.edu
    creation_date:          2020-05-27T20:54:03Z
    data_specs_version:     01.00.31
    experiment:             SSP-based RCP scenario with high radiative forcin...
    experiment_id:          ssp585
    forcing_index:          1
    fr

In [17]:
total_area = ds_area.areacella.sum(dim=['lon', 'lat'])
ta_timeseries = (ds.tas * ds_area.areacella).sum(dim=['lon', 'lat']) / total_area
ta_timeseries

<xarray.DataArray (time: 1032)>
dask.array<shape=(1032,), dtype=float32, chunksize=(600,)>
Coordinates:
  * time     (time) object 2015-01-15 12:00:00 ... 2100-12-15 12:00:00

In [21]:
import nc_time_axis
import cftime

In [25]:
import random
d_time = [cftime.Datetime360Day(year=2017, month=2, day=n) for n in range(1, 31)]
temperatures = [round(random.uniform(0, 12), 3) for _ in range(len(d_time))]

plt.plot(d_time, temperatures)
plt.margins(0.1)
plt.ylim(0, 12)
plt.xlabel("Date")
plt.ylabel("Temperature")
plt.show()

TypeError: __init__() takes at least 3 positional arguments (1 given)